In [1]:
import streamlit as st
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.agents import Tool, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain_ollama import OllamaLLM
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


In [ ]:
pip install streamlit

In [2]:
print("Is CUDA available?", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device Name:", torch.cuda.get_device_name(0))
else:
    print("No GPU device found. Using CPU.")


Is CUDA available? True
Device Name: NVIDIA GeForce RTX 2060


In [3]:
pdf_path = "D:\\Downloads\\Agent_langchain\\1561986011.General Catalogue.pdf"
loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()


In [4]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
)


C:\Users\Abdallah\AppData\Local\Temp\ipykernel_2176\2721713610.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [5]:
vectorstore = FAISS.from_documents(pages, embeddings)


In [6]:
def search_pdf(query: str) -> str:
    search_results = vectorstore.similarity_search(query, k=6)
    if not search_results:
        return "Sorry, I couldn't find any relevant information in the catalog."

    result_text = "Here are the results that match your query:\n\n"
    for idx, result in enumerate(search_results):
        result_text += f"{idx + 1}. {result.page_content}\n\n"
    return result_text

search_tool = Tool(
    name="Product Search",
    func=search_pdf,
    description="Use this tool to search within the provided PDF document."
)


In [7]:
agent_prompt = """
You are a friendly and knowledgeable sales agent who helps customers find information from a catalog-like document. 
Your task is to answer user queries based on the content of the provided PDF document.

The document contains various sections with information about products, features, and other relevant details.

When a user asks a question, try to find the most relevant sections of the document that answer their query and present them in a concise and helpful manner.

If no direct match is found, provide the closest related content or offer suggestions based on the available document sections.

If you don't have any relevant information to answer the user's query, let them know politely that you couldn't find anything.
"""

# Initialize the Ollama LLM
llm = OllamaLLM(model="qwen2.5-coder")

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent_with_memory = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent_type="zero-shot-react-description",
    memory=memory,
    verbose=True,
    max_iterations=6,
    handle_parsing_errors=True,
    prompt=agent_prompt,
    temperature=0
)

C:\Users\Abdallah\AppData\Local\Temp\ipykernel_2176\1208998924.py:17: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
C:\Users\Abdallah\AppData\Local\Temp\ipykernel_2176\1208998924.py:19: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent_with_memory = initialize_agent(


In [8]:
retrieval_agent_prompt = """
You are a retrieval agent. Your only task is to use the provided 'Product Search' tool to retrieve information relevant to the user's query. 
Do not interpret, summarize, or finalize the answer. Just return the content from the tool. 
If nothing is relevant, return what the tool returns.
"""

final_agent_prompt = """
You are a knowledgeable and friendly sales agent.
You have retrieved the following relevant sections from the document:
{retrieved_content}

Now, based on the user's query: "{user_query}", provide a concise and helpful answer. 
If the retrieved content does not directly answer the question, provide the best possible response based on the given information.
If no relevant information is found, politely say you couldn't find anything.
"""


In [9]:
# Initialize LLM
llm = OllamaLLM(model="qwen2.5-coder:7b-instruct")

# Agent 1: Retrieval agent (uses the search tool to get relevant text)
retrieval_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
retrieval_agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent_type="zero-shot-react-description",
    memory=retrieval_memory,
    verbose=True,
    max_iterations=3,
    handle_parsing_errors=True,
    prompt=retrieval_agent_prompt,
    temperature=0.0 # Lower temperature for more deterministic retrieval
)

# Agent 2: Final answer agent (uses only the LLM and no tools)
final_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# Note: Since final_agent doesn't use tools, we can directly call LLM.run()
# We'll just format the prompt and pass it to llm without initialize_agent for simplicity.
def generate_final_answer(retrieved_content: str, user_query: str) -> str:
    prompt = final_agent_prompt.format(retrieved_content=retrieved_content, user_query=user_query)
    return llm(prompt)

In [12]:
query = "which of the chairs is the most relaxing chair?"
retrieved_content = retrieval_agent.run(query)
final_answer = generate_final_answer(retrieved_content, query)
print(final_answer)




> Entering new AgentExecutor chain...
To determine which chair is the most relaxing chair, I should search within the provided PDF document for information on different types of chairs and their relaxation qualities.

Action: Product Search
Action Input: "relaxing chairs"
Observation: Here are the results that match your query:

1. 142 143
BLOSS 
CHAIRS
BLOSS is an elegantly mini-
malist ergonomic ofﬁce chair, 
completely in translucent 
mesh, featuring automatic 
back tension adjustment and 
a horizontally adjustable seat, 
with optional clip-on fabric or 
leather covers for the seat and 
backrest. The well thought-
out mesh concept ensures 
optimal support of the user’s 
body while seated. All frame 
elements are recessed to 
avoid any discomfort for the 
user as his/her body is only 
supported by the mesh.
BLOSS works with a specially developed mechanism that is regulated 
automatically by the user’s weight. No spring pressure regulation is required. 
The seat inclination adjustme

C:\Users\Abdallah\AppData\Local\Temp\ipykernel_2176\2282574382.py:24: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm(prompt)


I apologize for any inconvenience, but as an AI model, I don't have access to real-time data or personal experiences. Therefore, I'm unable to determine which chair is the most relaxing based on your query. However, I can suggest some factors to consider when looking for a comfortable chair, such as ergonomic design, adjustable settings, and plush cushioning. If you provide more specific details about what you're looking for in a relaxing chair, I'd be happy to assist further!
